# Football Match Prediction - Prediction Upload

This notebook predicts the next matchday using the trained classifier and encoder, then uploads the predictions to the betting platform.


In [1]:
import pickle
import pandas as pd
import numpy as np

# Load datasets and models
training_df = pickle.load(open("data/prepped_match_df.pck", "rb"))
next_matchday_df = pickle.load(open("data/next_matchday_df.pck", "rb"))
classifier = pickle.load(open("classifier_20250927_174216.pck", "rb"))
encoder = pickle.load(open("encoder.pck", "rb"))

print(f"Data loaded: {len(next_matchday_df)} matches to predict")


Data loaded: 9 matches to predict


In [2]:
# Prepare features for prediction
next_matchday_df_prep = next_matchday_df.drop(["date", "teamHomeName", "teamAwayName", "teamHomeRank", "teamAwayRank"], axis=1)
prediction_features = next_matchday_df_prep.drop(['resultClass'], axis=1)

# Make predictions
predictions_encoded = classifier.predict(prediction_features)
prediction_probabilities = classifier.predict_proba(prediction_features)
predictions_decoded = encoder.inverse_transform(predictions_encoded)

# Get probability for each predicted class
prediction_probabilities_max = []
for i, pred_encoded in enumerate(predictions_encoded):
    class_index = np.where(classifier.classes_ == pred_encoded)[0][0]
    prob = prediction_probabilities[i, class_index]
    prediction_probabilities_max.append(prob)

# Create results dataframe
results_df = pd.DataFrame({
    'Date': next_matchday_df['date'],
    'Home_Team': next_matchday_df['teamHomeName'],
    'Away_Team': next_matchday_df['teamAwayName'],
    'Matchday': next_matchday_df['matchDay'],
    'Season': next_matchday_df['season'],
    'Prediction': predictions_decoded,
    'Prediction_Probability': prediction_probabilities_max,
    'Home_Value': next_matchday_df['teamHomeValue'],
    'Away_Value': next_matchday_df['teamAwayValue']
})

results_df


,Date,Home_Team,Away_Team,Matchday,Season,Prediction,Prediction_Probability,Home_Value,Away_Value
9918,2025-12-19 20:30:00,Borussia Dortmund,Borussia Mönchengladbach,15,2025,1:1,0.149965,438.10,153.85
9919,2025-12-20 15:30:00,VfB Stuttgart,TSG Hoffenheim,15,2025,1:0,0.122492,277.63,154.85
9920,2025-12-20 15:30:00,VfL Wolfsburg,SC Freiburg,15,2025,1:1,0.197832,264.40,167.70
9921,2025-12-20 15:30:00,FC Augsburg,SV Werder Bremen,15,2025,1:1,0.126668,127.83,143.00
9922,2025-12-20 15:30:00,1. FC Köln,1. FC Union Berlin,15,2025,1:1,0.134353,74.68,124.40
9923,2025-12-20 15:30:00,Hamburger SV,Eintracht Frankfurt,15,2025,1:1,0.147203,80.35,336.40
9924,2025-12-20 18:30:00,RB Leipzig,Bayer 04 Leverkusen,15,2025,1:1,0.129695,422.75,439.75
9925,2025-12-21 15:30:00,1. FSV Mainz 05,FC St. Pauli,15,2025,0:1,0.129535,138.58,52.15
9926,2025-12-21 17:30:00,1. FC Heidenheim 1846,FC Bayern München,15,2025,0:1,0.114247,58.90,857.50


In [3]:
# Summary
print("=== MATCH PREDICTIONS ===")
for idx, row in results_df.iterrows():
    confidence = row['Prediction_Probability'] * 100
    print(f"{row['Home_Team']} vs {row['Away_Team']}")
    print(f"  → {row['Prediction']} (Confidence: {confidence:.1f}%)")
    print()


=== MATCH PREDICTIONS ===
Borussia Dortmund vs Borussia Mönchengladbach
  → 1:1 (Confidence: 15.0%)

VfB Stuttgart vs TSG Hoffenheim
  → 1:0 (Confidence: 12.2%)

VfL Wolfsburg vs SC Freiburg
  → 1:1 (Confidence: 19.8%)

FC Augsburg vs SV Werder Bremen
  → 1:1 (Confidence: 12.7%)

1. FC Köln vs 1. FC Union Berlin
  → 1:1 (Confidence: 13.4%)

Hamburger SV vs Eintracht Frankfurt
  → 1:1 (Confidence: 14.7%)

RB Leipzig vs Bayer 04 Leverkusen
  → 1:1 (Confidence: 13.0%)

1. FSV Mainz 05 vs FC St. Pauli
  → 0:1 (Confidence: 13.0%)

1. FC Heidenheim 1846 vs FC Bayern München
  → 0:1 (Confidence: 11.4%)



In [4]:
# Upload predictions to betting platform
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from difflib import get_close_matches
import dotenv
import os

# Load environment variables
dotenv.load_dotenv(".env")

# Start browser and login
driver = webdriver.Chrome()
driver.get(os.getenv("LINK-TIPPABGABE"))

# Login process
nav_button = driver.find_element(By.CLASS_NAME, "navtoggle")
nav_button.click()

email_input = driver.find_element(By.ID, "kennung")
email_input.send_keys(os.getenv("EMAIL"))
passwort_input = driver.find_element(By.ID, "passwort")
passwort_input.send_keys(os.getenv("PASSWORT"))

login_button = driver.find_element(By.NAME, "submitbutton")
login_button.click()


In [8]:
# Handle cookie banner if present
try:
    frame = driver.switch_to.frame(driver.find_element(By.XPATH, "/html/body/div[2]/iframe"))
    driver.find_element(By.XPATH, "/html/body/div[2]/div/div/div/div[3]/div[1]/div[2]/button").click()
    driver.switch_to.default_content()
except:
    print("No cookie banner found or already handled")
    driver.switch_to.default_content()


In [6]:
# Fill in predictions
driver.get(os.getenv("LINK-TIPPABGABE"))
tippabgabe_tabelle = driver.find_element(By.ID, "tippabgabeSpiele")
for datarow in tippabgabe_tabelle.find_elements(By.CLASS_NAME, "datarow"):
    homeTeam = datarow.find_element(By.CLASS_NAME, "col1").get_attribute("innerHTML")
    if homeTeam != "":
        # Find matching team in our predictions
        selectedTeam = get_close_matches(homeTeam, results_df["Home_Team"])[0]
        prediction = results_df[results_df["Home_Team"] == selectedTeam]["Prediction"].iloc[0].split(":")
        
        # Fill in the prediction
        inputs = datarow.find_elements(By.TAG_NAME, "input")
        inputs[1].clear()
        inputs[1].send_keys(prediction[0])
        inputs[2].clear()
        inputs[2].send_keys(prediction[1])
        
        print(f"Filled: {selectedTeam} -> {prediction[0]}:{prediction[1]}")

print("All predictions filled!")


Filled: Borussia Dortmund -> 1:1
Filled: VfL Wolfsburg -> 1:1
Filled: VfB Stuttgart -> 1:0
Filled: FC Augsburg -> 1:1
Filled: 1. FC Köln -> 1:1
Filled: Hamburger SV -> 1:1
Filled: RB Leipzig -> 1:1
Filled: 1. FSV Mainz 05 -> 0:1
Filled: 1. FC Heidenheim 1846 -> 0:1
All predictions filled!


In [7]:
# Submit predictions (optional - uncomment to actually submit)
driver.find_element(By.NAME, "submitbutton").click()
print("Predictions submitted!")

ElementClickInterceptedException: Message: element click intercepted: Element <button type="submit" name="submitbutton">...</button> is not clickable at point (266, 916). Other element would receive the click: <iframe src="https://cdn.privacy-mgmt.com/index.html?hasCsp=true&amp;message_id=1230027&amp;consentUUID=e28a6fa3-d2e6-421c-a853-b92968100376&amp;consent_origin=https%3A%2F%2Fcdn.privacy-mgmt.com%2Fconsent%2Ftcfv2&amp;preload_message=true&amp;concatenatedUUID=e28a6fa3-d2e6-421c-a853-b92968100376~~~&amp;version=v1&amp;consentLanguage=de" id="sp_message_iframe_1230027" title="SP Consent Message"></iframe>
  (Session info: chrome=143.0.7499.148); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#elementclickinterceptedexception
Stacktrace:
0   chromedriver                        0x0000000102914b2c cxxbridge1$str$ptr + 3033596
1   chromedriver                        0x000000010290cb30 cxxbridge1$str$ptr + 3000832
2  

In [ ]:
# Close browser
driver.close()
print("Browser closed.")


Browser closed.
